YOLOv5n

In [ ]:
try:
    print("Cloning YOLOv5 repository and installing dependencies...")
    !git clone https://github.com/ultralytics/yolov5.git
    %cd yolov5
    !pip install -r requirements.txt
    print("YOLOv5 repository cloned and dependencies installed successfully.")
except Exception as e:
    print(f"Error while cloning repository or installing dependencies: {e}")

Cloning YOLOv5 repository and installing dependencies...
Cloning into 'yolov5'...
remote: Enumerating objects: 17075, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (41/41), done.
remote: Total 17075 (delta 29), reused 30 (delta 12), pack-reused 17022 (from 1)
Receiving objects: 100% (17075/17075), 15.69 MiB | 16.63 MiB/s, done.
Resolving deltas: 100% (11724/11724), done.
/content/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.8/898.8 kB 58.5 MB/s eta 0:00:00
YOLOv5 repository cloned and dependencies installed successfully.


In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Import necessary libraries
try:
    import json
    import os
    import cv2
    import random
    import shutil
    from shutil import copyfile
    from yolov5.models.common import DetectMultiBackend
    import torch
    from pathlib import Path
    from PIL import Image
    print("Required libraries imported successfully.")
except Exception as e:
    print(f"Error while importing libraries: {e}")

Required libraries imported successfully.


In [ ]:
# Define paths for dataset and output directory
images_folder = '/content/drive/MyDrive/Colab Notebooks/val2017'
json_file = '/content/drive/MyDrive/Colab Notebooks/instances_val2017.json'
output_dir = '/content/drive/MyDrive/Colab Notebooks/yolov5_dataset'

# Validation to ensure paths exist
try:
    assert os.path.exists(images_folder), f"Images folder does not exist: {images_folder}"
    assert os.path.exists(json_file), f"JSON file does not exist: {json_file}"
    assert os.path.exists(output_dir), f"Output directory does not exist: {output_dir}"
    print("Paths validated successfully.")
except Exception as e:
    print(f"Error in paths validation: {e}")

In [ ]:
# Prepare output directories for training, validation, and testing
try:
    os.makedirs(f'{output_dir}/train/images', exist_ok=True)
    os.makedirs(f'{output_dir}/train/labels', exist_ok=True)
    os.makedirs(f'{output_dir}/val/images', exist_ok=True)
    os.makedirs(f'{output_dir}/val/labels', exist_ok=True)
    os.makedirs(f'{output_dir}/test/images', exist_ok=True)
    os.makedirs(f'{output_dir}/test/labels', exist_ok=True)
    print("Output directories created successfully.")
except Exception as e:
    print(f"Error while creating output directories: {e}")

In [ ]:
# Load COCO annotations and map category IDs
try:
    with open(json_file) as f:
        coco_data = json.load(f)

    # Find categories present in annotations
    annotation_category_ids = {anno['category_id'] for anno in coco_data['annotations']}
    filtered_categories = [cat for cat in coco_data['categories'] if cat['id'] in annotation_category_ids]

    # Sort categories by ID and map them to continuous indices
    category_mapping = {cat['id']: i for i, cat in enumerate(filtered_categories)}
    print(f"Category mapping completed with {len(category_mapping)} categories.")
except Exception as e:
    print(f"Error while processing COCO annotations: {e}")

In [ ]:
# Define function to convert COCO bbox to YOLO format
def coco_to_yolo(bbox, image_width, image_height):
    try:
        x, y, w, h = bbox
        x_center = (x + w / 2) / image_width
        y_center = (y + h / 2) / image_height
        width = w / image_width
        height = h / image_height
        return [x_center, y_center, width, height]
    except Exception as e:
        print(f"Error while converting COCO bbox to YOLO format: {e}")
        return None

In [ ]:
# Split dataset into training, validation, and test sets
try:
    random.seed(42)
    images = coco_data['images']
    random.shuffle(images)

    # Using 80-10-10 split for 5000 images
    total_images = len(images)
    train_images = images[:int(0.8 * total_images)]
    val_images = images[int(0.8 * total_images):int(0.9 * total_images)]
    test_images = images[int(0.9 * total_images):]

    print(f"Dataset split into {len(train_images)} training, {len(val_images)} validation, and {len(test_images)} test images.")
except Exception as e:
    print(f"Error while splitting dataset: {e}")

In [ ]:
# Process and save images and annotations
def process_images(image_subset, split_name):
    try:
        processed_count = 0  # Count successfully processed images
        skipped_files = []  # Track skipped files for better debugging

        for image_info in image_subset:
            image_id = image_info['id']
            image_filename = image_info['file_name']
            image_path = os.path.join(images_folder, image_filename)

            # Check if image exists and read dimensions
            if not os.path.exists(image_path):
                skipped_files.append(f"{image_filename}: File does not exist.")
                continue

            image = cv2.imread(image_path)
            if image is None:
                skipped_files.append(f"{image_filename}: Image could not be loaded (possibly corrupted).")
                continue

            image_height, image_width, _ = image.shape

            # Create label file path
            label_file_path = os.path.join(output_dir, split_name, 'labels', image_filename.replace('.jpg', '.txt'))

            # Process annotations for this image
            annotations = [anno for anno in coco_data['annotations'] if anno['image_id'] == image_id]
            with open(label_file_path, 'w') as label_file:
                for anno in annotations:
                    if anno['category_id'] not in category_mapping:
                        skipped_files.append(f"{image_filename}: Skipping category_id {anno['category_id']} (not in mapping).")
                        continue  # Skip unmapped category IDs

                    # Convert bbox to YOLO format
                    bbox = anno['bbox']
                    yolo_bbox = coco_to_yolo(bbox, image_width, image_height)
                    if yolo_bbox is None:
                        skipped_files.append(f"{image_filename}: Skipping bbox for annotation ID {anno['id']} due to conversion error.")
                        continue

                    # Write annotation to label file
                    category_id = category_mapping[anno['category_id']]
                    label_file.write(f"{category_id} {' '.join(map(str, yolo_bbox))}\n")

            # Copy the image to the split's images directory (Automatically creates missing folders)
            output_image_path = os.path.join(output_dir, split_name, 'images', image_filename)

            # Ensure the target folder exists, create if not
            os.makedirs(os.path.dirname(output_image_path), exist_ok=True)

            # Copy the image from the original directory to the new directory
            copyfile(image_path, output_image_path)

            processed_count += 1

        # Log skipped files
        if skipped_files:
            print(f"Skipped {len(skipped_files)} files during {split_name} processing:")
            for msg in skipped_files:
                print(f"  - {msg}")

        print(f"Processed {processed_count}/{len(image_subset)} images for the {split_name} split.")
    except Exception as e:
        print(f"Error while processing images for {split_name} split: {e}")

# Process all splits
process_images(train_images, 'train')
process_images(val_images, 'val')
process_images(test_images, 'test')

In [ ]:
# Create dataset.yaml for YOLOv5
try:
    names = [cat['name'] for cat in filtered_categories]
    yaml_content = f"""
    train: /content/drive/MyDrive/Colab Notebooks/yolov5_dataset/train/images
    val: /content/drive/MyDrive/Colab Notebooks/yolov5_dataset/val/images
    test: /content/drive/MyDrive/Colab Notebooks/yolov5_dataset/test/images

    nc: {len(names)}
    names: {names}
    """
    with open('/content/drive/MyDrive/Colab Notebooks/yolov5_dataset/dataset.yaml', 'w') as f:
        f.write(yaml_content)
    print("Dataset.yaml file created.")
except Exception as e:
    print(f"Error while creating dataset.yaml: {e}")


In [ ]:
# Disable WANDB for YOLOv5 (optional for Colab)
os.environ["WANDB_MODE"] = "disabled"

In [ ]:
# Load YOLOv5 model and initiate training
try:
    print("Loading YOLOv5 model...")
    model_path = '/content/drive/MyDrive/Colab Notebooks/yolov5_dataset/yolov5n_model/weights/best.pt'
    model = DetectMultiBackend(model_path)  # Load the pre-trained model or previously trained model
    print("YOLOv5 model loaded successfully.")
except Exception as e:
    print(f"Error while loading YOLOv5 model: {e}")

In [ ]:
# Train YOLOv5 model with optimizations
try:
    print("Training YOLOv5 model...")
    !python /content/yolov5/train.py --img 640 --batch 16 --epochs 5 --data '/content/drive/MyDrive/Colab Notebooks/yolov5_dataset/dataset.yaml' \
                                     --weights '/content/drive/MyDrive/Colab Notebooks/yolov5_dataset/yolov5n_model/weights/best.pt' \
                                     --project '/content/drive/MyDrive/Colab Notebooks/yolov5_dataset' --name yolov5n_model --workers 4
    print("Training completed successfully.")
except Exception as e:
    print(f"Error during training: {e}")

In [ ]:
# Evaluate YOLOv5 model
try:
    print("Evaluating YOLOv5 model...")
    !python /content/yolov5/val.py --data "/content/drive/MyDrive/Colab Notebooks/yolov5_dataset/dataset.yaml" \
                                   --weights "/content/drive/MyDrive/Colab Notebooks/yolov5_dataset/yolov5n_model/weights/best.pt" --img 640
    print("Evaluation completed successfully.")
except Exception as e:
    print(f"Error during evaluation: {e}")

In [ ]:
# Test the trained YOLOv5 model
try:
    print("Testing YOLOv5 model...")
    model_path = f'/content/drive/MyDrive/Colab Notebooks/yolov5_dataset/yolov5n_model/weights/best.pt'
    model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)

    # Perform inference
    image_path = '/content/img 11.jpg'  # Update with a test image path
    results = model(image_path)

    # Display results
    results.show()
    print("Testing completed successfully.")
except Exception as e:
    print(f"Error during testing: {e}")

In [ ]:
# Resume training for additional epochs
try:
    print("Resuming YOLOv5 training...")
    !python /content/yolov5/train.py --img 640 --batch 16 --epochs 10 \
                                     --data '/content/drive/MyDrive/Colab Notebooks/yolov5_dataset/dataset.yaml' \
                                     --weights '/content/drive/MyDrive/Colab Notebooks/yolov5_dataset/yolov5n_model/weights/last.pt' \
                                     --resume
    print("Training resumed successfully.")
except Exception as e:
    print(f"Error while resuming training: {e}")

Resuming YOLOv5 training...
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2024-12-06 09:34:44.725937: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-06 09:34:44.761322: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-06 09:34:44.771668: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: (1) Create a W&B account
wandb: (2) Use an existing W&B ac

Pickle File

In [ ]:
import torch
import pickle
from yolov5.models.common import DetectMultiBackend

# Load YOLOv5 model from the .pt file
pt_model_path = '/content/drive/MyDrive/Colab Notebooks/yolov5_dataset/yolov5n_model/weights/best.pt'
pkl_model_path = '/content/drive/MyDrive/Colab Notebooks/yolov5_dataset/yolov5n_model/weights/best.pkl'

try:
    print("Loading YOLOv5 model from .pt file...")
    model = DetectMultiBackend(pt_model_path)  # Load model with YOLOv5-specific backend
    state_dict = model.model.state_dict()  # Extract the state dictionary
    print("Model loaded and state dictionary extracted.")

    # Save state dictionary as a .pkl file
    print("Saving model state dictionary to .pkl file...")
    with open(pkl_model_path, 'wb') as f:
        pickle.dump(state_dict, f)
    print(f"Model successfully saved to {pkl_model_path}.")
except Exception as e:
    print(f"Error during conversion: {e}")

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Loading YOLOv5 model from .pt file...


Fusing layers... 
Model summary: 157 layers, 1867405 parameters, 0 gradients, 4.5 GFLOPs


Model loaded and state dictionary extracted.
Saving model state dictionary to .pkl file...
Model successfully saved to /content/drive/MyDrive/Colab Notebooks/yolov5_dataset/yolov5n_model/weights/best.pkl.


In [ ]:
import cv2
import torch
import numpy as np
from ultralytics.yolo.utils.ops import letterbox

try:
    print("Loading state dictionary from .pkl file...")
    with open(pkl_model_path, 'rb') as f:
        loaded_state_dict = pickle.load(f)

    # Initialize a new YOLOv5 model and load the state dictionary
    print("Initializing new model and loading state dictionary...")
    new_model = DetectMultiBackend(pt_model_path)  # Use the same architecture as the original model
    new_model.model.load_state_dict(loaded_state_dict)
    print("State dictionary loaded successfully into the new model.")

    # Verify performance by running inference
    test_image_path = '/content/img 11.jpg'  # Update with an actual test image path
    assert os.path.exists(test_image_path), f"Test image not found: {test_image_path}"

    print("Performing inference...")

    # Load and preprocess the image
    image = cv2.imread(test_image_path)  # Load the image using OpenCV
    if image is None:
        raise ValueError(f"Unable to load image at {test_image_path}")

    # Convert the image from BGR (OpenCV format) to RGB (model's expected format)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Resize and pad the image to the model's input size using letterbox function
    resized_image, _, _ = letterbox(image, new_shape=(640, 640))  # Resize to 640x640 with padding
    resized_image_tensor = torch.from_numpy(resized_image).float().permute(2, 0, 1) / 255.0  # Normalize and convert to tensor
    resized_image_tensor = resized_image_tensor.unsqueeze(0)  # Add batch dimension

    # Perform inference
    results = new_model(resized_image_tensor)
    results.show()  # Display results directly (e.g., in a Jupyter Notebook or Colab environment)
    print("Verification and inference completed successfully.")
except AssertionError as e:
    print(f"Assertion Error: {e}")
except ValueError as e:
    print(f"Value Error: {e}")
except Exception as e:
    print(f"Error during verification: {e}")

ModuleNotFoundError: No module named 'ultralytics.yolo'